In [ ]:
import numpy as np
import seaborn
import torch
import torch.nn as nn
import torchvision.datasets
import torchvision.models
import torchvision.transforms
import time
import functools
import itertools
import random
import matplotlib.pyplot as plt
from torch.multiprocessing import Pool,freeze_support
from scipy.linalg import hadamard

from torch.optim.optimizer import Optimizer, required
import pickle

In [ ]:
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return itertools.chain.from_iterable(itertools.combinations(s, r) 
                                         for r in range(len(s)+1))


max_nbhdsize = 12
x_S_binary_strings = {}
hadamards = {}
for r in range(max_nbhdsize + 1):
    K = 2 ** r
    printing_spec = '0' + str(r) + 'b'
    x_S_binary_strings[K] = torch.tensor([list(map(int,list(format(x_S, printing_spec)))) 
                    for x_S in range(0,K)]).cuda()
    
    #note: may want to keep huge hadamard matrices on cpu to save gpu memory
    hadamards[K] = torch.tensor(hadamard(K)).float().cuda()
del K
del printing_spec

def visualize(image):
    shaped_image = np.matrix(image).reshape(28,28)
    seaborn.heatmap(shaped_image)

# often used internal subroutine
def compute_probs_list(S,x_S_binaries,X_probabilities_list):
    if len(S) == 0:
        # avoid annoying indexing issue in the normal code
        probs_list = torch.ones(X_probabilities_list.shape[0], 1).cuda()
    else:
        big_list = X_probabilities_list[:,S,:].unsqueeze(1).expand(-1,x_S_binaries.shape[0],-1,-1)
        idx = x_S_binaries.unsqueeze(2).expand(big_list.shape[0],-1,-1,1)
        big_list_inner = torch.gather(big_list, 3, idx).squeeze(3)
        probs_list = torch.prod(big_list_inner,axis=2)
    return probs_list


def conditional_variances(i, S, train_data_centered, X_probabilities_list):
    # compute E[(X_i - E[X_i | X_S,X_j])^2] for every j
    n = train_data_centered[0].numel() 
    K = 2 ** len(S) 

    x_S_binaries = x_S_binary_strings[K]
                    
    probs_list = compute_probs_list(S,x_S_binaries,X_probabilities_list)
    #probs_list = torch.prod(
    #    torch.cat([X_probabilities_list[:,S,x_S_binary].unsqueeze(1) 
    #                for x_S_binary in x_S_binaries],dim=1),
    #                        axis=2)
    
    cond_means = torch.einsum('a,aj,aik->ijk', train_data_centered[:,i],#2 * train_data[:,i] - 1, 
                              probs_list, X_probabilities_list)
    normalization = torch.einsum('aj,aik->ijk', probs_list, 
                                 X_probabilities_list)

    normalization[normalization == 0] = 1
    cond_means = cond_means/normalization
            
    predictions = torch.einsum('aj,ijk,aik -> ai', probs_list, cond_means,
                                    X_probabilities_list)
    errs = torch.sum((train_data_centered[:,i].unsqueeze(1).expand(-1,n) - predictions) ** 2,axis=0)
    return errs/train_data_centered.shape[0]

    
def find_neighborhood(i,k,train_data_centered,X_probabilities_list,term_eps,prog_eps=0.01):
    S = [] 
    mean = torch.mean(train_data_centered[:,i])
    variance = 1 - mean ** 2
    # the above is an overestimate when we have fractional pixels, so this is always legitimate
    if variance < term_eps: 
        return S
    last_variance = variance

    for step in range(k):
        cond_variances = conditional_variances(i,S,train_data_centered,X_probabilities_list)
        sorted_variances, indices = torch.sort(cond_variances)
        for j in indices:
            if j not in S and j != i:
                if cond_variances[j] > (1 - prog_eps) * last_variance:
                    return S # not enough progress
                last_variance = cond_variances[j]
                S.append(j.item())
                if cond_variances[j] < term_eps:
                    return S
                else:
                    break
    print(i)
    return S



In [ ]:
# clamp_eps shouldn't be too small because of floating point precision requirements
def atanh(x,clamp_eps=1e-4):
    x = torch.clamp(x, min=-1 + clamp_eps, max=1 - clamp_eps)
    return 0.5 * (torch.log(1 + x) - torch.log(1 - x))

def compute_single_coeff(nbhds, X_probabilities_list, centered_data, i):
    S = nbhds[i]
    K = 2 ** len(S)
    x_S_binaries = x_S_binary_strings[K]
    
    probs_list = compute_probs_list(S,x_S_binaries,X_probabilities_list)
                                    #torch.tensor(x_S_binaries).cuda(),X_probabilities_list)                

    cond_means = centered_data @ probs_list 
    normalization = torch.sum(probs_list,dim=0) 
    normalization[normalization == 0] = 1
    cond_means = atanh(cond_means/normalization)
    hadam = hadamards[K]

    coeffs = (hadam @ cond_means)/K      
    return coeffs 

def compute_coeffs(train_data, nbhds):
    n = train_data[0].numel() 
    fourier_coeffs = []
    train_data_centered = 2 * train_data - 1
    X_probabilities_list = torch.cat((train_data.unsqueeze(2), 
                                      1 - train_data.unsqueeze(2)), axis=2)
    fourier_coeffs = [compute_single_coeff(nbhds, X_probabilities_list, train_data_centered[:,i], i) for i in range(n)]
    return fourier_coeffs

In [ ]:
DATA_FOLDER = "./data"
CONTENT_FOLDER = "./data/content_mnist"

CUDA = torch.cuda.is_available()
CUDA_DEVICE = 0

# after training once, can be used to skip some parts of the code
LOAD_FROM_FILE = False

if CUDA:
    torch.cuda.set_device(CUDA_DEVICE)
    print("Using CUDA")


print('Loading dataset...')


#unused routines which can be used with other methods. binarization slightly improves accuracy

tensor_and_normalize = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(), 
     torchvision.transforms.Normalize((0.1307,), (0.3081,))])

tensor_and_binarize = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(), 
    lambda x: x > 0.45,
    lambda x: x.float(),])

# Choose MNIST or FashionMNIST here
#dataset = torchvision.datasets.FashionMNIST
dataset = torchvision.datasets.MNIST

transform = torchvision.transforms.ToTensor()

train_dataset_unnormalized = dataset(root=DATA_FOLDER, train=True,
                                     transform=transform,
                                     download=True)

test_dataset_unnormalized = dataset(root=DATA_FOLDER, train=False,
                                    transform=transform,
                                    download=True)

VISIBLE_UNITS = train_dataset_unnormalized[0][0].numel()  # was 28 x 28 images

In [ ]:
neighborhoods = []
fourier_coefficients = []
NUM_CURRENT_CLASS = 10 # for now

if LOAD_FROM_FILE:
    f = open(CONTENT_FOLDER + "neighborhoods.pkl",'rb')
    neighborhoods = pickle.load(f)
    f.close()

    f = open(CONTENT_FOLDER + "fourier_coefficients.pkl",'rb')
    fourier_coefficients = pickle.load(f)
    f.close()

else:
    for class_num in range(NUM_CURRENT_CLASS):
        print('Class', class_num)
        dset_train = torch.utils.data.dataset.Subset(train_dataset_unnormalized,
                                                    np.where(torch.tensor(train_dataset_unnormalized.targets)==class_num)[0])
        nbhds = []
        datapts = list(dset_train)

        # can be set smaller to speed up training.
        actual_use_size = len(datapts)
        train_data = [dset_train[i][0].flatten().unsqueeze(0) for i in range(actual_use_size)]#torch.cat(batch_list[0:8])
        train_data = torch.cat(train_data)
        if CUDA: 
            train_data  = train_data.cuda() #much faster on gpu than cpu
        train_data_centered = 2 * train_data - 1

        X_probabilities_list = torch.cat((train_data.unsqueeze(2), 
                                        1 - train_data.unsqueeze(2)), axis=2)
        fnbhd = functools.partial(find_neighborhood,k=max_nbhdsize,
                                    train_data_centered=train_data_centered,
                                    X_probabilities_list=X_probabilities_list,
                                    term_eps=0.01
                                        # 0.001 is a conservative choice (seems TOO SMALL, 90% on fmnist-2), was using 0.005 (93%)
                                        # 0.01 gave 92% on mnist with nbhd=12, 0.005 did as well.
                                )
        del train_data_centered
        del X_probabilities_list

        print("Computing nbhds")
        if CUDA:
            nbhds = list(map(fnbhd, range(VISIBLE_UNITS)))
        else:
            p = Pool(4)
            nbhds = list(p.map(fnbhd, range(VISIBLE_UNITS)))

        neighborhoods.append(nbhds)
        print("Computing coeffs")
        fourier = compute_coeffs(train_data, nbhds)
        fourier_coefficients.append(fourier)
        del train_data

    f = open(CONTENT_FOLDER + "neighborhoods.pkl","wb")

    pickle.dump(neighborhoods,f)

    f.close()

    f = open(CONTENT_FOLDER + "neighborhoods.txt","w")
    for i in range(NUM_CURRENT_CLASS):
        f.write('Class '+ str(i) + '\n')
        for k in range(VISIBLE_UNITS):
            f.write(str(k) + ': ' + str(neighborhoods[i][k]) + '\n')
    f.close()

    f = open(CONTENT_FOLDER + "fourier_coefficients.pkl","wb")

    pickle.dump(fourier_coefficients,f)

    f.close()

    f = open(CONTENT_FOLDER + "fourier_coefficients.txt","w")
    for i in range(NUM_CURRENT_CLASS):
        f.write('Class '+ str(i) + '\n')
        for k in range(VISIBLE_UNITS):
            f.write(str(k) + ': ' + str(fourier_coefficients[i][k]) + '\n')
    f.close()



In [ ]:
#torch.autograd.set_detect_anomaly(True) #set to True for debugging

JUST_LINEAR = False # do a normal logistic regression, for testing purposes
USE_INDIVIDUAL = True # logistic regression on the generated "evidence weights"
RESTRICTED_COEFFS = False # force some of the logistic regression coeffs = 0. unused

class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, num_classes, neighbors, fourier_coeffs):
        super(LogisticRegression, self).__init__()
        if JUST_LINEAR:
            self.linear = torch.nn.Linear(input_dim,num_classes)
        elif RESTRICTED_COEFFS:
            self.node_weighting = torch.nn.Parameter(torch.ones(num_classes, input_dim).cuda())
            self.linear = torch.nn.Linear(num_classes * input_dim, num_classes)
            self.linear.weight.requires_grad = False 
            self.linear.weight.zero_()
        elif USE_INDIVIDUAL:
            self.linear = torch.nn.Linear(num_classes * input_dim, num_classes)
        else:
            self.linear = torch.nn.Linear(num_classes, num_classes)
            self.linear.weight.requires_grad = False 
            self.linear.weight.zero_()
    
        if CUDA:
            self.linear = self.linear.cuda()

        self.neighbors = neighbors
        self.fourier_coeffs = fourier_coeffs
        self.num_classes = num_classes

        # compute x_S_binaries, x_S_sizes
        self.x_S_binaries = []
        self.x_S_sizes = []
        for k in range(self.num_classes):
            S = self.neighbors[k]
            x_S_binaries_k = []
            x_S_sizes_k = []
            for i in range(len(S)):
                K = 2 ** len(S[i]) 
                x_S_binaries_ki = x_S_binary_strings[K]
                x_S_binaries_k.append(x_S_binaries_ki)
                
                # this doesn't work without the .float(), it does integer division...
                # maybe because normal python doesn't distinguish integer and float clearly enough?
                x_S_sizes_k.append(1/(torch.sum(x_S_binaries_ki,dim=1).float() + 1))

            self.x_S_binaries.append(x_S_binaries_k)
            self.x_S_sizes.append(x_S_sizes_k)

    # calls compute_probs_list 
    def compute_probs_list(self,i,k,X_probabilities_list):
        return compute_probs_list(self.neighbors[k][i],self.x_S_binaries[k][i],X_probabilities_list)

    def compute_expectation(self, x, i, k, X_probabilities_list):
        probs_list = self.compute_probs_list(i,k,X_probabilities_list)
        """Compute E[X_i | X_{\sim i}]"""
        return torch.tanh(torch.einsum('ai,i->a', 
                            probs_list, self.fourier_coeffs[k][i]))
        
   
    def compute_fn_unnormalized(self, x, i, k, X_probabilities_list):
        """Compute the sum of fourier coefficients involving node i"""
        probs_list = self.compute_probs_list(i,k,X_probabilities_list)
        return torch.einsum('a,ai,i->a', 
                            2 * x[:,i] - 1, probs_list,  self.fourier_coeffs[k][i])
     

    def compute_fn_normalized(self, x, i, k, X_probabilities_list):
        """Compute the sum of fourier coefficients involving node i,
           normalizing coefficients by 1/support size"""
        probs_list = self.compute_probs_list(i,k,X_probabilities_list)
        # we use self.xs_sizes[k][i] to renormalize, fixing overcounting of higher order
        # monomials
        return torch.einsum('a,ai,i,i->a', 
                            2 * x[:,i] - 1, probs_list, 
                            self.x_S_sizes[k][i], self.fourier_coeffs[k][i])
    
    # only outputs sums
    def precompute(self, x):
        n = x[0].numel()
        outputs = torch.zeros(size=(x.shape[0],self.num_classes)).cuda()
        X_probabilities_list = torch.cat((0. * x.unsqueeze(2) + 1, 2 * x.unsqueeze(2) - 1), axis=2)
        for k in range(self.num_classes):
            outputs[:,k] += sum([self.compute_fn_normalized(x, i, k, 
                                                  X_probabilities_list) for i in range(n)])
        return outputs

    # outputs individual node statistics
    def precompute_individual(self, x):
        n = x[0].numel()
        outputs = torch.zeros(size=(x.shape[0],self.num_classes,n)).cuda()
        X_probabilities_list = torch.cat((0. * x.unsqueeze(2) + 1, 2 * x.unsqueeze(2) - 1), axis=2)
        for k in range(self.num_classes):
            for i in range(n):
                outputs[:,k,i] = self.compute_fn_normalized(x, i, k, 
                                                  X_probabilities_list)
        return outputs    
    
    def forward(self, x_precomputed):
        if RESTRICTED_COEFFS:
            outputs = torch.sum(self.node_weighting.expand(x_precomputed.shape[0],-1,-1) 
                                * x_precomputed.reshape(-1,self.node_weighting.shape[0],self.node_weighting.shape[1]),
                                dim=2)
            outputs = outputs + self.linear(x_precomputed) # add biases 
        elif not USE_INDIVIDUAL:
            outputs = self.linear(x_precomputed) + x_precomputed # self.beta * x_precomputed
        else:
            outputs = self.linear(x_precomputed)
        return outputs

    def mf_onestep(self, x, k):
        """Run one step of mean-field iteration on data points in x according to class k"""
        n = x[0].numel()
        output = x.clone()
        X_probabilities_list = torch.cat((0. * x.unsqueeze(2) + 1, 2 * x.unsqueeze(2) - 1), axis=2)
        for i in range(n):
            output[:,i] = (1 + self.compute_expectation(x,i,k,X_probabilities_list))/2
        return output

    def gibbs(self, x, k, num_steps):
        """Run Gibbs sampling num_steps on data points in x according to class k"""
        n = x[0].numel()
        for step in range(num_steps):
            X_probabilities_list = torch.cat((0. * x.unsqueeze(2) + 1, 2 * x.unsqueeze(2) - 1), axis=2)
            i = random.randrange(n)
            ps = (1 + self.compute_expectation(x,i,k,X_probabilities_list))/2
            randomness = torch.from_numpy(np.random.random_sample(size = len(ps))).cuda()
            x[:,i] = (ps > randomness)
        return x

    def mf_closest_class(self, x):
        errs = []
        for k in range(NUM_CURRENT_CLASS):
            errs.append(torch.sum((self.mf_onestep(x,k) - x) ** 2,axis=1).unsqueeze(1))
        errs = torch.cat(errs,axis=1)
        return torch.argmin(errs, dim=1)


from torch.autograd import Variable

if NUM_CURRENT_CLASS < 10:
    dset_train = torch.utils.data.dataset.Subset(train_dataset_unnormalized,
                                             np.where(train_dataset_unnormalized.targets < NUM_CURRENT_CLASS)[0])
    dset_test = torch.utils.data.dataset.Subset(test_dataset_unnormalized,
                                            np.where(test_dataset_unnormalized.targets < NUM_CURRENT_CLASS)[0])
else:
    dset_train = train_dataset_unnormalized
    dset_test = test_dataset_unnormalized

epochs = 100
lr_rate = 100

model = LogisticRegression(VISIBLE_UNITS, NUM_CURRENT_CLASS, neighborhoods, fourier_coefficients)
criterion = torch.nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.LBFGS(model.parameters(), lr=lr_rate, line_search_fn="strong_wolfe", max_iter=4, history_size=10)

In [ ]:
#need a smaller batch size for larger nbhds to fit in memory                                             
batch_size = 2 * 8192 

train_loader = torch.utils.data.DataLoader(dataset=dset_train, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=dset_test, batch_size=batch_size, shuffle=False)

precomputed_train = []
precomputed_test = []

if LOAD_FROM_FILE:
    precomputed_train = torch.load(CONTENT_FOLDER + 'precompute_train.pt')
    precomputed_test = torch.load(CONTENT_FOLDER + 'precompute_test.pt')
else:
    for i, (images, labels) in enumerate(train_loader):
        print(i)
        images = images.view(-1, 28 * 28).cuda()
        if USE_INDIVIDUAL:
            outputs = model.precompute_individual(images)
        else:
            outputs = model.precompute(images)
        precomputed_train.append(outputs)

    for i, (images, labels) in enumerate(test_loader):
        print(i)
        images = images.view(-1, 28 * 28).cuda()
        if USE_INDIVIDUAL:
            outputs = model.precompute_individual(images)
        else:
            outputs = model.precompute(images)
        precomputed_test.append(outputs)

    MERGE_SETS = True # always True, changes stuff to tensors
    if MERGE_SETS:
        precomputed_train = [torch.cat(precomputed_train)]
        precomputed_test = [torch.cat(precomputed_test)]

        batch_size = len(dset_train)
        train_loader = torch.utils.data.DataLoader(dataset=dset_train, batch_size=batch_size, shuffle=False)
        test_loader = torch.utils.data.DataLoader(dataset=dset_test, batch_size=batch_size, shuffle=False)

    # normalize logistic regression features for slightly faster training
    NORMALIZE = True 
    if NORMALIZE:
        feature_means = torch.mean(precomputed_train[0],dim=0,keepdim=True)
        precomputed_train[0] -= feature_means.expand_as(precomputed_train[0])
        feature_stddev = torch.sqrt(torch.mean(precomputed_train[0] ** 2,dim=0,keepdim=True))
        
        # NOTE: due to precision issues, means are not exactly zero. we must clamp small stddevs!
        # setting it to 1 prevents an issue
        feature_stddev[feature_stddev < 1e-4] = 1

        precomputed_train = [precomputed_train[0]/feature_stddev.expand_as(precomputed_train[0])]
        precomputed_test = [(precomputed_test[0] - feature_means.expand_as(precomputed_test[0]))/feature_stddev.expand_as(precomputed_test[0])]

    torch.save(precomputed_train, CONTENT_FOLDER + 'precompute_train.pt')
    torch.save(precomputed_test, CONTENT_FOLDER + 'precompute_test.pt')

    # flatten
    if USE_INDIVIDUAL:
        precomputed_train[0] = precomputed_train[0].view(precomputed_train[0].shape[0],-1)
        precomputed_test[0] = precomputed_test[0].view(precomputed_test[0].shape[0],-1)


batch_size = len(dset_train)
train_loader = torch.utils.data.DataLoader(dataset=dset_train, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=dset_test, batch_size=batch_size, shuffle=False)

In [ ]:
### Training loop for logistic regression.

#LOAD_FROM_FILE = False # reenable this training loop
if LOAD_FROM_FILE:
    model.load_state_dict(torch.load(CONTENT_FOLDER + 'model.pth'))
else:
    for epoch in range(int(epochs)):
        print(model.linear.bias.data)
        for i, (images, labels) in enumerate(train_loader):
            print(epoch)
            labels = labels.cuda()
            if JUST_LINEAR:
                precomputed = images.cuda().view(-1,28 * 28)
            else:
                precomputed = precomputed_train[i]

            def closure():
                optimizer.zero_grad()
                outputs = model(precomputed)
                loss = criterion(outputs,labels)
                loss.backward()

                # compute training accuracy
                _, predicted = torch.max(outputs.data, 1)
                correct = (predicted == labels.cuda()).sum().float()
                training_accuracy = 100 * correct.item()/labels.shape[0]
                print(training_accuracy)
                return loss

            loss = optimizer.step(closure)
            VERBOSE = False
            if epoch%5==0 and VERBOSE:
                # calculate Accuracy
                correct = 0
                total = 0
                for j, (images, labels) in enumerate(test_loader):
                    if JUST_LINEAR:
                        precomputed = images.cuda().view(-1,28 * 28)#.unsqueeze(2).view(-1,28 * 28 * 10)
                    else:
                        precomputed = precomputed_test[j]
                    outputs = model(precomputed).cuda()
                    _, predicted = torch.max(outputs.data, 1)
                    
                    total+= labels.size(0)
                    for k in range(10):
                        print("Number predicted for ", k, ": ", (predicted == k).sum().item())
                        print("Number actual for ", k, ": ", (labels == k).sum().item())
                    # for gpu, bring the predicted and labels back to cpu fro python operations to work
                    correct+= (predicted == labels.cuda()).sum()
                accuracy = 100 * float(correct)/total
                print("Epoch: {}. Training Loss: {}. Test Accuracy: {}.".format(epoch,loss.item(), accuracy))

    torch.save(model.state_dict(), CONTENT_FOLDER + 'model.pth')

In [ ]:
### Fun neighborhood visualization
import matplotlib.pyplot as plt

for class_num in range(NUM_CURRENT_CLASS):
    z = np.zeros(VISIBLE_UNITS)
    for nbhd in neighborhoods[class_num]:
        for node in nbhd:
            z[node] += 1
    plt.imshow(z.reshape(28,28), cmap='hot', interpolation='nearest')
    plt.show()

In [ ]:
#### Generate samples by Gibbs sampling
plt.figure()
num_examples = 5
f, axarr = plt.subplots(num_examples,NUM_CURRENT_CLASS)
for k in range(NUM_CURRENT_CLASS):
    x = torch.rand(num_examples, 28 * 28).cuda()
    gibbs_result = model.gibbs(x, k, 6000)
    smoothed = model.mf_onestep(gibbs_result,k)
    for i in range(num_examples):
        axarr[i,k].axis('off')
        axarr[i,k].imshow((smoothed[i].cpu().view(28,28)),cmap='gray')        
plt.show()

In [ ]:
#### Generate samples from TRAINING SET for comparison
plt.figure()
num_examples = 5
f, axarr = plt.subplots(num_examples,NUM_CURRENT_CLASS)
for k in range(NUM_CURRENT_CLASS):
    # rejection sampling
    for i in range(num_examples):
        while True:
            idx = random.randrange(0,len(train_dataset_unnormalized))
            if train_dataset_unnormalized[idx][1] == k:
                smoothed = train_dataset_unnormalized[idx][0].squeeze()
                break
        axarr[i,k].axis('off')
        axarr[i,k].imshow((smoothed.cpu().view(28,28)),cmap='gray')        
plt.show()